In [1]:
import pandas as pd
import numpy as np
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_file, show
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.cosmology import z_at_value

In [2]:
alldb = pd.read_pickle('../dataframes/fitdb/allfits_1D.pkl')
morph_db = alldb.query('W_UVJ == "Q" and AGN != "AGN" and lmass >= 10.5 and n_f < 3 and Re < 20 ')

bspec = [27458,294464,36348,48631,19290,32566,32691,33093,26272,35640,45333, 30144]
# nog141 = [27915,37955,17746,17735]
nog102 = [27714,37189,26139,32799,47223,22774,28890,23073,31452,24033]
# nog102 = []

inout = []
for i in morph_db.index:     
    if morph_db.id[i] not in bspec and morph_db.id[i] not in nog102: 
        inout.append('i')
    else:
        inout.append('o')
        
morph_db['inout'] = inout
mdb = morph_db.query('inout == "i" and 0.7 < zgrism < 2.5 and Sigma1 > 10**9.6')

/Users/Vince.ec/miniconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
p = figure(plot_width=800, plot_height=600, title=None)
p.scatter(np.log10(mdb.query('Sigma1 > 10**10').Sigma1), 
          cosmo.lookback_time(mdb.query('Sigma1 > 10**10').z_50), 
          size=10, color="navy", alpha=0.5, name='Compact')

# create a new plot and share both ranges
p.scatter(np.log10(mdb.query('Sigma1 < 10**10').Sigma1), 
          cosmo.lookback_time(mdb.query('Sigma1 < 10**10').z_50), 
            size=10, color="firebrick", alpha=0.5, name='Extended')

p.legend.location = "top_left"
p.legend.click_policy="hide"

# output_file("interactive_legend.html", title="interactive_legend.py example")
output_file("lines.html")

show(p)